# Constraining FaIR

Replicate WGI Chapter 7 method based on notebooks/160_WG3_constrain_fair_samples.ipynb
at https://github.com/IPCC-WG1/Chapter-7
and apply the alternative MH sampler

The results from the Chapter 7 code are required
and here assumed to be available as the following HDF5 file.
```
./datain/fair_samples_unconstrained.h5
```

Output
- `dataout/constraining_fair_indicators.nc` Indicators used in the Chapter 7 method
- `dataout/constraining_fair_accept.nc` Results from the Chapter 7 constraining
- `dataout/constraining_fair_indicators_alt.nc` Indicators used in the MH sampler
- `dataout/constraining_fair_mhout.h5` Results from the MH constraining

In [1]:
import json
import numpy as np
import pandas as pd
import scipy.stats as stats
import openpyxl
import h5py
from netCDF4 import Dataset

In [2]:
from src.util import RetrieveGitHub, retrieve_url, dffilter, df2nc
from src.stats import sampling, asymmetric_gaussian

In [3]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

## Reference data

In [4]:
path = 'data_input/observations/AR6 FGD assessment time series - GMST and GSAT.xlsx'
path = repo_ch7.retrieve(path)

[2024-07-22 08:34:49 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/observations/AR6 FGD assessment time series - GMST and GSAT.xlsx retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/observations/AR6%20FGD%20assessment%20time%20series%20-%20GMST%20and%20GSAT.xlsx on 2024-06-20


In [5]:
wb = openpyxl.load_workbook(path, read_only=True, data_only=True)
ws = wb['GMST data sets and changes']
rows = ws.iter_rows(
    min_row=2, # 1-based index
    max_row=2+(2020 - 1850 + 1),
    min_col=12,
    max_col=20,
    values_only=True,
)
columns = next(rows)
df_gmst_obs = (
    pd.DataFrame(list(rows), columns=('Year',) + columns[1:])
    .dropna(axis=1)
    .set_index('Year')
)
wb.close()

In [6]:
path = 'data_input/observations/AR6_OHC_ensemble_FGDprelim.csv'
path = repo_ch7.retrieve(path)

[2024-07-22 08:34:54 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/observations/AR6_OHC_ensemble_FGDprelim.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/observations/AR6_OHC_ensemble_FGDprelim.csv on 2024-06-20


In [7]:
df_ohc_obs = pd.read_csv(path, skiprows=1, index_col=0)

## Original constraining

### 1. GSAT 1995-2014

In [8]:
f_in = h5py.File('./datain/fair_samples_unconstrained.h5')

In [9]:
idx_year = pd.Index(f_in['/historical/year'][:])

In [10]:
# The followings are redundant, but do so for consistency with the previous code
dset = f_in['/historical/T']
data = np.zeros(dset.shape[::-1])
data[:] = dset[:].T

In [11]:
slc = slice(idx_year.get_loc(1850), idx_year.get_loc(1900)+1)
gsat = data - data[slc].mean(axis=0)

In [12]:
slc = slice(idx_year.get_loc(1850), idx_year.get_loc(2014) + 1)
diff = gsat[slc] - df_gmst_obs.loc[1850:2014, '4-set mean'].values[:, None]
rmse_temp = np.sqrt((diff ** 2).sum(axis=0) / diff.shape[0])

In [13]:
rmse_temp_crit = 0.135
accept_temp = rmse_temp < rmse_temp_crit
accept_temp.sum()

311968

In [14]:
df_ind = pd.DataFrame({'temp': rmse_temp})
df_accept = pd.DataFrame({'temp': accept_temp})
gatts = {'rmse_temp_crit': rmse_temp_crit}

### 2. Ocean heat uptake from 1971 to 2018

In [15]:
d_ohu = pd.Series(
    f_in['/historical/OHU'][:, idx_year.get_loc(2018)]
    - f_in['/historical/OHU'][:, idx_year.get_loc(1971)]
) * 1e-21

In [16]:
name_central = 'Central Estimate Full-depth'
name_unc = 'Full-depth Uncertainty (1-sigma)'
d_ohu_obs = (df_ohc_obs.loc[2018.5, name_central] - df_ohc_obs.loc[1971.5, name_central])
d_ohu_obs_unc = np.sqrt(
    df_ohc_obs.loc[1971.5, name_unc]**2 + df_ohc_obs.loc[2018.5, name_unc]**2
)

In [17]:
ohu_rate = 0.90 # Use an assumed factor of 0.90
accept_ohu = np.logical_and(
    ohu_rate * d_ohu > (d_ohu_obs - d_ohu_obs_unc),
    ohu_rate * d_ohu < (d_ohu_obs + d_ohu_obs_unc),
)
accept_ohu.sum()

322738

In [18]:
df_ind['ohu'] = d_ohu
df_accept['ohu'] = accept_ohu
gatts.update({
    'ohu_rate': ohu_rate,
    'ohu_obs': d_ohu_obs,
    'ohu_obs_unc': d_ohu_obs_unc,
})

### 3. CO2 concentrations in 2014

In [19]:
fn = 'rcmip-concentrations-annual-means-v5-1-0.csv'
path = retrieve_url(
    f'./datain/rcmip/{fn}',
    f'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/{fn}',
)

[2024-07-22 08:37:34 src.util] INFO:Use local file datain/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-concentrations-annual-means-v5-1-0.csv on 2024-06-20


In [20]:
df = pd.read_csv(path)

In [21]:
id_vars = df.columns[:7].to_list()
df = df.set_index(id_vars).rename(columns=int)

In [22]:
co2_obs_2014 = dffilter(
    df,
    Scenario='ssp245',
    Region='World',
    Variable=lambda x: x.endswith('CO2'),
).loc[:, 2014].squeeze()
co2_obs_2014

397.5469792683919

In [23]:
d_co2_2014 = pd.Series(f_in['/historical/C_CO2'][:, idx_year.get_loc(2014)])

In [24]:
co2_obs_2014_unc = 0.36
accept_co2 = np.logical_and(
    d_co2_2014 > co2_obs_2014 - co2_obs_2014_unc,
    d_co2_2014 < co2_obs_2014 + co2_obs_2014_unc,
)
accept_co2.sum()

21560

In [25]:
df_ind['co2'] = d_co2_2014
df_accept['co2'] = accept_co2
gatts.update({
    'co2_obs': co2_obs_2014,
    'co2_obs_unc': co2_obs_2014_unc,
})

### 4. Airborne fraction

In [26]:
df_accept['temp_ohu_co2'] = pd.Series(
    df_accept['temp'].values
    * df_accept['ohu'].values
    * df_accept['co2'].values
)

In [27]:
df_accept['temp_ohu_co2'].sum()

3751

In [28]:
d1 = df_accept['temp_ohu_co2']
accept_inds = d1.loc[d1].index.values
len(accept_inds)

3751

In [29]:
path = repo_ch7.retrieve('data_input/random_seeds.json')

[2024-07-22 08:38:11 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/random_seeds.json retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/random_seeds.json on 2024-06-12


In [30]:
with path.open() as f1:
    SEEDS = json.load(f1)

In [31]:
d_af140 = pd.Series(f_in['/1pctCO2/af140'][:])

In [32]:
SAMPLES = len(d_af140)
SAMPLES

1000000

In [33]:
accept_prob = stats.uniform.rvs(loc=0, scale=1, size=SAMPLES, random_state=SEEDS[79])
norm_af140 = stats.norm(loc=.597, scale=.049)
pdf_ref = norm_af140.pdf(0.597)

accept_af = (
    norm_af140.pdf(d_af140.loc[accept_inds].values) / pdf_ref
    >= accept_prob[:len(accept_inds)]
)

In [34]:
accept_af.sum()

2237

In [35]:
d1 = df_accept['temp_ohu_co2'].copy()
d1.loc[d1] = accept_af
d1.sum()

2237

In [36]:
df_ind['af'] = d_af140
df_ind['accept_prob'] = accept_prob
df_accept['af_combined'] = d1

The following is not in the Chapter 7 approach, but included for a consistent comparison.

In [37]:
df_accept['af_single'] = pd.Series(norm_af140.pdf(d_af140.values) / pdf_ref >= accept_prob)

In [38]:
f_in.close()

## Alternative constraining

In [39]:
slc = slice(idx_year.get_loc(1995), idx_year.get_loc(2014) + 1)
df_ind_alt = {
    'temp': gsat[slc].mean(axis=0),
    'ohu': pd.Series(d_ohu * ohu_rate),
    'co2': d_co2_2014,
    'af': d_af140,
}

NINETY_TO_ONESIGMA = stats.norm.ppf(0.95)

map_rv = {
    'temp': # GSAT 1995-2014
    pd.Series(asymmetric_gaussian(0.85, (0.67, 0.98))),
    'ohu': # OHU 2018 relto 1971
    stats.norm(loc=396.0, scale=0.5*(506.2 - 285.7)/NINETY_TO_ONESIGMA),
    'co2': # CO2 2014
    # Use the RCMIP central value of 397.547 ppm for consistency
    # instead of 397.1 ppm
    stats.norm(loc=co2_obs_2014, scale=0.4/NINETY_TO_ONESIGMA),
    'af': # AF 140
    norm_af140,
}

# Additional constraint
slc = slice(idx_year.get_loc(1961), idx_year.get_loc(1990) + 1)
df_ind_alt['temp_trend'] = df_ind_alt['temp'] - gsat[slc].mean(axis=0)
map_rv['temp_trend'] = (
    stats.norm(
        loc=0.85 - 0.36,
        scale=np.sqrt(
            asymmetric_gaussian(0.36, (0.22, 0.45)).var(ddof=1)
            + asymmetric_gaussian(0.85, (0.67, 0.98)).var(ddof=1)
        ),
    )
)

df_ind_alt = pd.DataFrame(df_ind_alt)

In [40]:
# ensure unbiased variance 
map_rv['temp'].var(), map_rv['temp'].var(ddof=1)

(0.008907285310997317, 0.008907285310997317)

In [41]:
def wrap_sampling(df, rv, seed=0, maxlen=10000, maxout=2000):
    if df.ndim == 1:
        mean = rv.mean()
        sig2 = rv.var()
        ret = sampling(df, mean, sig2, seed=seed, maxlen=maxlen)
    else:
        mean = np.array([rv[k].mean() for k in df])
        sig2 = np.array([rv[k].var() for k in df])
        # Target covariance is assumed to be scaled from input covariance
        # with the ratio of target variance to input variance
        cov = df.cov().values
        scale = np.sqrt(sig2 / cov.diagonal())
        scale = scale * scale.reshape((-1, 1))
        ret = sampling(df, mean, cov * scale, seed=seed, maxlen=maxlen)

    mhout = pd.Series(df.index[ret])
    member = (
        mhout.loc[np.random.randint(0, len(df), maxout)]
        .reset_index(drop=True)
    )

    return mhout, member

### Individual constraints

In [42]:
mhout = {}
member_rand = {}

len_input = 20000

for k, v in df_ind_alt.items():
    _mhout, _member = wrap_sampling(v.iloc[:len_input], map_rv[k])
    mhout[f'{k}__single'] = _mhout
    member_rand[f'{k}__single'] = _member

[2024-07-22 08:40:29 src.stats] INFO:acceptance rate 0.36235
[2024-07-22 08:40:33 src.stats] INFO:acceptance rate 0.48
/home/tsutsui/samba/project/ar6-tlm/src/stats.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  if ft[ip] * fp[i] / (ft[i] * fp[ip]) > r:
[2024-07-22 08:40:37 src.stats] INFO:acceptance rate 0.0248
[2024-07-22 08:40:41 src.stats] INFO:acceptance rate 0.57245
[2024-07-22 08:40:45 src.stats] INFO:acceptance rate 0.71215


### GSAT constraints including trends

In [43]:
len_input = 20000
inds = ['temp', 'temp_trend']
_mhout, _member = wrap_sampling(df_ind_alt[inds].iloc[:len_input], map_rv)

[2024-07-22 08:41:33 src.stats] INFO:acceptance rate 0.18885


In [44]:
mhout['temp_&_temp_trend'] = _mhout
member_rand['temp_&_temp_trend'] = _member

### GSAT and OHU constraints

In [45]:
len_input = 50000
inds = ['temp', 'temp_trend', 'ohu']
_mhout, _member = wrap_sampling(df_ind_alt[inds].iloc[:len_input], map_rv)

[2024-07-22 08:41:57 src.stats] INFO:acceptance rate 0.08992


In [46]:
mhout['temp_&_temp_trend_&_ohu'] = _mhout
member_rand['temp_&_temp_trend_&_ohu'] = _member

### All constraints

First, roughly constrain input series in terms of CO2 2014. The central value is shifted to from 397.1 ppm to 397.55 ppm.

In [47]:
# use the central value from RCMIP (397.55 ppm)
co2_obs_2014, map_rv['co2'].ppf([0.05, 0.5, 0.95])

(397.5469792683919, array([397.14697927, 397.54697927, 397.94697927]))

In [48]:
scale_factor = 5.
rv = stats.norm(loc=co2_obs_2014, scale=0.4/NINETY_TO_ONESIGMA * scale_factor)
_mhout, _member = wrap_sampling(df_ind_alt['co2'], rv, maxout=100000)

[2024-07-22 08:45:41 src.stats] INFO:acceptance rate 0.11577


In [49]:
mhout['co2_pre'] = _mhout
member_rand['co2_pre'] = _member

Then, apply all the constraints.

In [50]:
_mhout, _member = wrap_sampling(
    df_ind_alt.iloc[member_rand['co2_pre']].reset_index(drop=True),
    map_rv, maxout=10000,
)

[2024-07-22 08:47:10 src.stats] INFO:acceptance rate 0.02832


In [51]:
mhout['all'] = _mhout
member_rand['all'] = _member

## Save results

In [53]:
path_out = './dataout/constraining_fair_indicators.nc'
df2nc(
    path_out,
    df_ind.rename_axis('Member').rename_axis(columns='Variable'),
    {}, gatts=gatts,
)

ncf = Dataset(path_out, 'r+')

for name, d1 in df_ind.items():
    ncf.variables[name][:] = d1.values

ncf.close()

/home/tsutsui/samba/project/ar6-tlm/src/util.py:158: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  fill_value=getattr(data, 'fill_value', None),
[2024-07-22 08:48:55 src.util] INFO:dataout/constraining_fair_indicators.nc is created


In [54]:
df_accept

,temp,ohu,co2,temp_ohu_co2,af_combined,af_single
0,False,True,False,False,False,False
1,False,False,False,False,False,True
2,False,True,False,False,False,True
3,False,False,False,False,False,False
4,True,True,False,False,False,False
...,...,...,...,...,...,...
999995,True,False,False,False,False,True
999996,True,True,False,False,False,True
999997,True,False,False,False,False,True
999998,True,True,False,False,False,True


In [55]:
df_accept = df_accept.astype('int8')
df_accept

,temp,ohu,co2,temp_ohu_co2,af_combined,af_single
0,0,1,0,0,0,0
1,0,0,0,0,0,1
2,0,1,0,0,0,1
3,0,0,0,0,0,0
4,1,1,0,0,0,0
...,...,...,...,...,...,...
999995,1,0,0,0,0,1
999996,1,1,0,0,0,1
999997,1,0,0,0,0,1
999998,1,1,0,0,0,1


In [56]:
path_out = './dataout/constraining_fair_accept.nc'
df2nc(
    path_out,
    df_accept.rename_axis('Member').rename_axis(columns='Variable'),
    {},
)

ncf = Dataset(path_out, 'r+')

for name, d1 in df_accept.items():
    ncf.variables[name][:] = d1.values

ncf.close()

[2024-07-22 08:49:51 src.util] INFO:dataout/constraining_fair_accept.nc is created


In [57]:
path_out = './dataout/constraining_fair_indicators_alt.nc'
df2nc(
    path_out,
    df_ind_alt.rename_axis('Member').rename_axis(columns='Variable'),
    {}, gatts=gatts,
)

ncf = Dataset(path_out, 'r+')

for name, d1 in df_ind_alt.items():
    ncf.variables[name][:] = d1.values

ncf.close()

[2024-07-22 08:50:49 src.util] INFO:dataout/constraining_fair_indicators_alt.nc is created


In [62]:
path_out = './dataout/constraining_fair_mhout.h5'
f1 = h5py.File(path_out, 'w')

In [63]:
grp = f1.create_group('mhout')

for k, v in mhout.items():
    grp.create_dataset(k, data=v.values)

In [64]:
grp = f1.create_group('member')

for k, v in member_rand.items():
    grp.create_dataset(k, data=v.values)

In [65]:
f1.close()